# RewardFunction5
```
Key Elements:
1. Using Bézier curves to determine curvature of straight versus curved track
2. Determined cut-off curvature value as 1e-4
   - If track curvature (determined by calculating kappa) is greater than 1e-4, then the track is curved. If kappa is 
   less than 1e-4, track is straight
3. Rewarding centered to line and speed differently based on curvature of track
```

In [2]:
def reward_function(params):
    '''
    Uses Bezier curve math to determine curvature
    '''
   
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    progress = params['progress']/100
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params['steps']
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 150
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.8 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete.  if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    elif progress < .1:
        #assuming that the car starts on a straight track, set the base reward to be the maximum
        #we will adjust things later on
        reward = REWARD_MAX
    elif steps > 0:        # we want the vehicle to continue making progress
        reward = REWARD_MAX * max(progress, 0.4)
    
    # Determine waypoints
    prev_pt = waypoints[closest_waypoints[0]] # closest waypoint behind car
    next_pt = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    nextnext_pt = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    curveCalculation = True
    
    # Determining curvature
    myNumerator = (next_pt[0] - prev_pt[0])*(prev_pt[0] - 2 * next_pt[0] + nextnext_pt[0]) + (next_pt[1] - prev_pt[1])*(prev_pt[1] - 2 * next_pt[1] + nextnext_pt[1])
    myDenominator = (prev_pt[0] - 2 * next_pt[0] + nextnext_pt[0])**2 + (prev_pt[1] - 2 * next_pt[1] + nextnext_pt[1])**2
    
    #assuming that myDenominator does not equal zero
    if (myDenominator != 0):
        curvatureTime = myNumerator / myDenominator
        x_prime = 2 * prev_pt[0] * (curvatureTime - 1) + 2 * next_pt[0] - 4 * next_pt[0] * curvatureTime + 2 * nextnext_pt[0] * curvatureTime
        x_double_prime = 2 * prev_pt[0] - 4 * next_pt[0] + 2 * nextnext_pt[0]
        y_prime = 2 * prev_pt[1] * (curvatureTime - 1) + 2 * next_pt[1] - 4 * next_pt[1] * curvatureTime + 2 * nextnext_pt[1] * curvatureTime
        y_double_prime = 2 * prev_pt[1] - 4 * next_pt[1] + 2 * nextnext_pt[1]   
        temp_value = (x_prime**2 + y_prime**2)**1.5
    
        #assuming that temp_value does not equal zero
        if temp_value != 0:
            curvature = abs(x_prime * y_double_prime - x_double_prime * y_prime) / temp_value
        else:
            curveCalculation = False
    else:
        curveCalculation = False
    
    # Find out whether track is straight or curved
    if curveCalculation:
        if curvature < .0001:
            # track is straight
            # reward speed on straight track
            reward += min(speed**2, 10)
            # Give higher reward if the car is closer to center line 
            if distance_from_center <= marker_1:
                reward += 5
            elif distance_from_center <= marker_2:
                reward += 1
            if steering < 15:
                reward += 5

        else:
            #track is curved, reward for being somewhat centered
            if distance_from_center <= marker_3:
                reward += 5
            if steering < 30:
                reward += 5 
            if speed > 2:
                reward += 3
            elif speed > 1:
                reward += 1
    else:
        #reward based on straight track criteria
        reward += min(speed**2, 10)
        # Give higher reward if the car is closer to center line 
        if distance_from_center <= marker_1:
            reward += 5
        elif distance_from_center <= marker_2:
            reward += 1
        if steering < 15:
            reward += 5
    
    return float(reward)
